In [ ]:
from azureml.core import Workspace, Model, Dataset
import subprocess
import requests
import json
import pandas as pd
import warnings
import requests
warnings.filterwarnings('ignore')

In [ ]:
ws = Workspace.from_config()
kv = ws.get_default_keyvault()

In [ ]:
acr_username = kv.get_secret('acr-username')
acr_password = kv.get_secret('acr-password')
acr_address = kv.get_secret('acr-address')

In [ ]:
container_name = 'manymodelsdeployment'
container_version = '<YOUR-CONTAINER-VERSION>'
location = f'{acr_address}/{container_name}:{container_version}'
location

In [ ]:
subprocess.check_call(['docker', 'login', acr_address, '-u', acr_username, '-p', acr_password])
subprocess.check_call(['docker', 'pull', location])
subprocess.check_call(['docker', 'run', '-d', '-p', '5001:5001', location])

In [ ]:
import shutil
try:
    shutil.rmtree('./tmp')
except Exception as e:
    pass

os.mkdir('./tmp')

testing_dataset = Dataset.get_by_name(ws, 'HomePrices_Testing_Data')
testing_dataset.download(target_path='./tmp')

In [ ]:
training_files = os.listdir('./tmp')

for file in training_files:
    file_name = os.path.basename(file)[:-4]
    model_name = file_name + '_MLModel'
    curr_df = pd.read_csv(f'./tmp/{file}')
    curr_df = curr_df.drop(columns=['target'])
    curr_df_string = curr_df.to_json(orient='records')
    body = {'model_name': model_name, 'data':curr_df_string}
    
    uri = 'http://localhost:5001/score'
    headers = {'Content-Type': 'application/json'}
    response = requests.post(uri, data=json.dumps(body), headers=headers)
    
    print(model_name)
    print(response.json())
    print()